<a href="https://colab.research.google.com/github/woo-seung/energyplus_colab/blob/master/energyplus_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EnergyPlus Python API tutorials
- [EnergyPlus python API documentation](https://nrel.github.io/EnergyPlus/api/python/index.html)
- [API full example](https://github.com/NREL/EnergyPlus/tree/e858cb2b02e0e681bc2492fbe3938c1e5d03a0d1/tst/EnergyPlus/api)
- [Input Output Reference v9.3.0](https://energyplus.net/sites/all/modules/custom/nrel_custom/pdfs/pdfs_v9.3.0/InputOutputReference.pdf)
- [EMS Application Guide v9.3.0](https://energyplus.net/sites/all/modules/custom/nrel_custom/pdfs/pdfs_v9.3.0/EMSApplicationGuide.pdf)

# 1. Install EnergyPlus

In [0]:
# install EP to "/usr/local/EnergyPlus-9-3-0"
!chmod +x /content/drive/My\ Drive/ep_drive/EnergyPlus-9.3.0-baff08990c-Linux-x86_64.sh
!sudo /content/drive/My\ Drive/ep_drive/EnergyPlus-9.3.0-baff08990c-Linux-x86_64.sh
# to capture C-level stdout/stderr pipes in Python
!pip install wurlitzer
# check EP
print('\n- Check EnergyPlus Version')
!energyplus -version

EnergyPlus, Copyright (c) 1996-2020, The Board of Trustees of the University of Illinois, The Regents of the University of California, through Lawrence Berkeley National Laboratory (subject to receipt of any required approvals from the U.S. Dept. of Energy), Oak Ridge National Laboratory, managed by UT-Battelle, Alliance for Sustainable Energy, LLC, and other contributors. All rights reserved.

NOTICE: This Software was developed under funding from the U.S. Department of Energy and the U.S. Government consequently retains certain rights. As such, the U.S. Government has been granted for itself and others acting on its behalf a paid-up, nonexclusive, irrevocable, worldwide license in the Software to reproduce, distribute copies to the public, prepare derivative works, and perform publicly and display publicly, and to permit others to do so.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

(1)

# 2. Initialization

In [0]:
# setting
%load_ext wurlitzer
import sys
import os
import shutil
sys.path.insert(0, '/usr/local/EnergyPlus-9-3-0')
from pyenergyplus.api import EnergyPlusAPI

# you should connect your google drive
OUTPUT_PATH = '/content/drive/My Drive/ep_drive/output'
if os.path.exists(OUTPUT_PATH): shutil.rmtree(OUTPUT_PATH)
if not os.path.exists(OUTPUT_PATH):os.makedirs(OUTPUT_PATH)

EPW_PATH = r'/content/drive/My Drive/ep_drive/epw/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw'
IDF_PATH = r'/content/drive/My Drive/ep_drive/idf/2ZoneDataCenterHVAC_wEconomizer.idf'
EP_CL = ['-d', OUTPUT_PATH, '-w', EPW_PATH, IDF_PATH]

# 3. Examples

## 1. Runtime API example

In [0]:
# Example 1

def dummy_callback_function():
    pass
api = EnergyPlusAPI()
api.runtime.callback_begin_new_environment(dummy_callback_function)
api.runtime.run_energyplus(EP_CL)

# api.runtime.clear_all_states() # ??
# api = EnergyPlusAPI()
# api.runtime.callback_begin_new_environment(dummy_callback_function)
# api.runtime.run_energyplus(EP_CL)

EnergyPlus Starting
EnergyPlus, Version 9.3.0-baff08990c, YMD=2020.06.04 12:31
Initializing Response Factors
Calculating CTFs for "EXT-WALLS", Construction # 1
Calculating CTFs for "FLOOR", Construction # 2
Calculating CTFs for "PARTITION", Construction # 4
Calculating CTFs for "PLENUM FLOOR", Construction # 5
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Factors
Computing Interior Diffuse Solar Absorption Factors
Computing Interior Diffuse Solar Exchange through Int

EnergyPlus Completed Successfully.


0

In [0]:
# Example 2

def environment_handler() -> None:
    print("OH HAI ENVIRONMENT")
    sys.stdout.flush()

def progress_handler(progress: int) -> None:
    if 49 < progress < 51:
        print("HALFWAY THERE!!")
        sys.stdout.flush()

def error_handler(message: bytes) -> None:
    if b'Warning' in message:
        print("GOT A WARNING UH OH!")
        sys.stdout.flush()

api = EnergyPlusAPI()
api.runtime.callback_begin_new_environment(environment_handler)
api.runtime.callback_progress(progress_handler)
api.functional.callback_error(error_handler)
api.runtime.run_energyplus(EP_CL)

## 2. Functional API example

In [0]:
# Example 1

api = EnergyPlusAPI()
glycol = api.functional.glycol(u"water")
cp = glycol.specific_heat(35.0)
refrigerant = api.functional.refrigerant("steam")
satPress = refrigerant.saturation_pressure(100.0)
psychrometrics = api.functional.psychrometrics()
rh = psychrometrics.relative_humidity_b(24, 0.009, 101325)
print(f'dry_bulb_temp: 24, humidity_ratio: 0.009, barometric_pressure: 101325 -> rh: {rh}')

In [0]:
# Example 2

api = EnergyPlusAPI()

# GLYCOL TESTS
glycol = api.functional.glycol(u"water")
for t in [5.0, 15.0, 25.0]:
    cp = glycol.specific_heat(t)
    rho = glycol.density(t)
    k = glycol.conductivity(t)
    viscosity = glycol.viscosity(t)
    print("Python API Test: Calculated properties at T=%s: %f, %f, %f, %f" % (t, cp, rho, k, viscosity))

# REFRIGERANT TESTS
refrigerant = api.functional.refrigerant("steam")
temperature = 100.0
satPress = refrigerant.saturation_pressure(temperature)  # expecting about 101325 Pa
thisPress = 100000
satTemp = refrigerant.saturation_temperature(thisPress)  # expecting about 100 degC
print("Python API Test: Saturated Properties: At 100C, Psat=%8.4f; at 100000Pa, Tsat=%8.4f" % (satPress, satTemp))
satLiqDens = refrigerant.saturated_density(temperature, 0.0)  # // liq = 958 kg/m3
satLiqCp = refrigerant.saturated_specific_heat(temperature, 0.0)  # liq = 4,216 J/kgK
satLiqEnth = refrigerant.saturated_enthalpy(temperature, 0.0)
print("C API Test: Sat Liq at 100C: rho=%8.4f, Cp=%8.4f, h=%8.4f" % (satLiqDens, satLiqCp, satLiqEnth))
satVapDens = refrigerant.saturated_density(temperature, 1.0)  # vap = 1/1.6718 ~~ 0.59 kg/m3
satVapCp = refrigerant.saturated_specific_heat(temperature, 1.0)  # vap = 2,080 J/kgK
satVapEnth = refrigerant.saturated_enthalpy(temperature, 1.0)
print("C API Test: Sat Vap at 100C: rho=%8.4f, Cp=%8.4f, h=%8.4f" % (satVapDens, satVapCp, satVapEnth))
enthDifference = satVapEnth - satLiqEnth  # vap-liq = 2,675,570-419,170 ~ 2,256,400 J/kg

# PSYCHROMETRIC TESTS
psychrometrics = api.functional.psychrometrics()
# // PSYCHROMETRICS
# // test point is:
# //   Barometric Pressure: 101325 Pa
# //   Dry Bulb Temp: 24 C
# //   Relative Humidity: 0.5
# //   Humidity Ratio: ~0.009 kg/kg
# //   Saturation Temp: ~17 C
# //   Saturation Pressure: 2985 Pa
# //   Enthalpy: ~48000 J/kg
# //   Specific Volume: 0.855 m3/kg
# //   Density: ~1.17
# //   Wet Bulb: ~17 C
# //   Dew Point: ~13 C
# //   Vapor Density: 0.0107 kg/m3
# //   Specific Heat: ~1007 J/kgK
print("Python API Test: Psych props, test point is about 101325Pa, 24C, 50%% humidity:")
db = psychrometrics.dry_bulb(48000, 0.009)
print("Python API Test: Expected DB ~ 24 C Calculated: %8.4f" % db)
rh = psychrometrics.relative_humidity(24, 0.0107)
rh2 = psychrometrics.relative_humidity_b(24, 0.009, 101325)
print("Python API Test: Expected RH ~ 0.5 Calculated: %8.4f, %8.4f" % (rh, rh2))
hr = psychrometrics.humidity_ratio(24, 48000)
hr2 = psychrometrics.humidity_ratio_b(13, 101325)
hr3 = psychrometrics.humidity_ratio_c(24, 0.5, 101325)
hr4 = psychrometrics.humidity_ratio_d(24, 17, 101325)
print("Python API Test: Expected HumRat ~ 0.009 Calculated: %8.4f, %8.4f, %8.4f, %8.4f" % (hr, hr2, hr3, hr4))
tSat = psychrometrics.saturation_temperature(48000, 101325)
print("Python API Test: Expected Tsat ~ 17 C Calculated: %8.4f" % tSat)
pSat = psychrometrics.saturation_pressure(24)
print("Python API Test: Expected Psat ~ 2985 Pa Calculated: %8.4f" % pSat)
h = psychrometrics.enthalpy(24, 0.009)
h2 = psychrometrics.enthalpy_b(24, 0.5, 101325)
print("Python API Test: Expected Enth ~ 0.48000 J/kg Calculated: %8.4f, %8.4f" % (h, h2))
volume = psychrometrics.specific_volume(24, 0.009, 101325)
print("Python API Test: Expected v ~ 0.855 m3/kg Calculated: %8.4f" % volume)
density = psychrometrics.density(101325, 24, 0.009)
print("Python API Test: Expected rho ~ 1.17 kg/m3 Calculated: %8.4f" % density)
wb = psychrometrics.wet_bulb(24, 0.009, 101325)
print("Python API Test: Expected WB ~ 17 C Calculated: %8.4f" % wb)
dp = psychrometrics.dew_point(0.009, 101325)
dp2 = psychrometrics.dew_point_b(24, 17, 101325)
print("Python API Test: Expected DP ~ 13 C Calculated: %8.4f, %8.4f" % (dp, dp2))
vaporDensity = psychrometrics.vapor_density(24, 0.009, 101325)
vaporDensity_2 = psychrometrics.vapor_density_b(24, 0.5)
print("Python API Test: Expected VapDensity ~ 0.0107 kg/m3 Calculated: %8.4f, %8.4f" % (vaporDensity, vaporDensity_2))
cp = psychrometrics.specific_heat(0.009)
print("Python API Test: Expected Cp ~ 1007 J/kgK Calculated: %8.4f" % cp)
energy = psychrometrics.latent_energy_of_air(24)
print("Python API Test: Calculated energy?: %8.4f" % energy)
moisture_energy = psychrometrics.latent_energy_of_moisture_in_air(24)
print("Python API Test: Calculated energy of moisture: %8.4f" % moisture_energy)

# check that we get error messages back:
error_count = 0


def error_handler(message: bytes) -> None:
    global error_count
    error_count += 1


api.functional.callback_error(error_handler)
erroneous_dew_point = psychrometrics.dew_point_b(16, 17, 101325)
print("Python API Test: Got back erroneous value of dew point: %8.4f\n" % erroneous_dew_point)
if error_count > 0:
    print("Python API Test: Errors were caught during dew point calculation, good!")
else:
    print("Python API Test: Errors were NOT caught during dew point calculation, bad!")
    sys.exit(1)

## 3. DataTransfer API example

In [0]:
# Example 1 - set outdoor dew point temperature to (oa_temp - 4) 

one_time = True
outdoor_temp_sensor = 0
outdoor_dew_point_actuator = 0

def time_step_handler():
    global one_time, outdoor_temp_sensor, outdoor_dew_point_sensor , outdoor_dew_point_actuator
    if one_time:
        if not api.exchange.api_data_fully_ready():
            return
        # get_variable_handle 입력값은 EP 출력값 중 .rdd에 나옴
        outdoor_temp_sensor = api.exchange.get_variable_handle( u"SITE OUTDOOR AIR DRYBULB TEMPERATURE", u"ENVIRONMENT")
        # get_actuator_handle 입력값은 EP 출력값 중 .edd에 나옴
        outdoor_dew_point_actuator = api.exchange.get_actuator_handle( "Weather Data", "Outdoor Dew Point", "Environment")
        one_time = False
    oa_temp = api.exchange.get_variable_value(outdoor_temp_sensor)
    api.exchange.set_actuator_value(outdoor_dew_point_actuator , oa_temp-4)

api = EnergyPlusAPI()
api.runtime.callback_end_zone_timestep_after_zone_reporting(time_step_handler)
api.exchange.request_variable("SITE OUTDOOR AIR DRYBULB TEMPERATURE", "ENVIRONMENT")
api.runtime.run_energyplus(EP_CL)

In [0]:
# Example 2

one_time = True
outdoor_temp_sensor = 0
outdoor_dew_point_sensor = 0
outdoor_dew_point_actuator = 0


def time_step_handler():
    global one_time, outdoor_temp_sensor, outdoor_dew_point_sensor, outdoor_dew_point_actuator
    sys.stdout.flush()
    if one_time:
        if api.exchange.api_data_fully_ready():
            # val = api.exchange.list_available_api_data_csv()
            # with open('/tmp/data.csv', 'w') as f:
            #     f.write(val.decode(encoding='utf-8'))
            outdoor_temp_sensor = api.exchange.get_variable_handle(
                u"SITE OUTDOOR AIR DRYBULB TEMPERATURE", u"ENVIRONMENT"
            )
            outdoor_dew_point_sensor = api.exchange.get_variable_handle(
                u"SITE OUTDOOR AIR DEWPOINT TEMPERATURE", u"ENVIRONMENT"
            )
            outdoor_dew_point_actuator = api.exchange.get_actuator_handle(
                "Weather Data", "Outdoor Dew Point", "Environment"
            )
            if outdoor_temp_sensor == -1 or outdoor_dew_point_sensor == -1 or outdoor_dew_point_actuator == -1:
                sys.exit(1)
            one_time = False
    api.exchange.set_actuator_value(outdoor_dew_point_actuator, -25)
    oa_temp = api.exchange.get_variable_value(outdoor_temp_sensor)
    print("Reading outdoor temp via getVariable, value is: %s" % oa_temp)
    dp_temp = api.exchange.get_variable_value(outdoor_dew_point_sensor)
    print("Actuated Dew Point temp value is: %s" % dp_temp)


api = EnergyPlusAPI()
api.runtime.callback_end_zone_timestep_after_zone_reporting(time_step_handler)
api.exchange.request_variable("SITE OUTDOOR AIR DRYBULB TEMPERATURE", "ENVIRONMENT")
api.exchange.request_variable("SITE OUTDOOR AIR DEWPOINT TEMPERATURE", "ENVIRONMENT")
api.runtime.run_energyplus(EP_CL)